## Import libraries

In [ ]:
from jsonschema import RefResolver, Draft7Validator, ValidationError
import json

## Functions

In [ ]:
def load_json(filepath: str) -> dict:
    return json.load(open(filepath, "r"))


def find_refs(obj, refs=None) -> list[str]:
    if refs is None:
        refs = []

    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == "$ref":
                refs.append(value[2:])
            else:
                find_refs(value, refs)

    return refs


def find_ref_schemas(schemas_filenames: list[str], refs=None):
    if refs is None:
        refs = []

    for schema_filename in schemas_filenames:
        schema = load_json(f"Metadata_Schemas_JSON_Schema/{schema_filename}")
        ref_schemas_filenames = find_refs(schema)

        if schema_filename not in refs:
            refs.append(schema_filename)
            find_ref_schemas(ref_schemas_filenames, refs)

    return refs


def set_up_validator(json_schema_filename):
    schemas_filenames = [json_schema_filename]
    all_schemas_filenames = find_ref_schemas(schemas_filenames)

    first_schema = load_json(f"Metadata_Schemas_JSON_Schema/{all_schemas_filenames[0]}")
    resolver = RefResolver(base_uri="http://example.com/", referrer=first_schema)

    for referenced_schema_filename in all_schemas_filenames:
        referenced_schema = load_json(
            f"Metadata_Schemas_JSON_Schema/{referenced_schema_filename}"
        )
        resolver.store[referenced_schema["$id"]] = referenced_schema

    validator = Draft7Validator(first_schema, resolver=resolver)

    return validator

## Prepare the JSON Schema validator

In [ ]:
validator = set_up_validator("molecule.schema.json")

## Validate the JSON file

In [ ]:
json_file = load_json("example.json")

try:
    validator.validate(instance=json_file)
    print("The JSON file is valid according to the schema.")
except ValidationError as e:
    message = e.schema["error_msg"] if "error_msg" in e.schema else e.message
    print(f"Invalid data: {message}")